# MNIST + CNN

코드 작성 순서
1. Module
2. parameter setting
3. Data => loader
4. Model
5.  Loss & Optimizer
6. Train
7. Test

In [0]:
# 1. Module
import torch
import torch.nn as nn
import torch.optim as optim

import torchvision.datasets as dsets
import torchvision.transforms as transforms

import torch.nn.init

In [0]:
# 2. parameter setting
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(777)
if device =='cuda':
    torch.cuda.manual_seed_all(777)
    
learning_rate = 0.001
training_epochs = 15
batch_size = 100

In [0]:
torch.cuda.is_available()

True

In [0]:
print(device)

cuda


In [0]:
# 3. Data
# MINST Train 데이터
mnist_train = dsets.MNIST(root='MNIST_data/',
                         train=True,
                         transform=transforms.ToTensor(),
                         download=True)

# MINST Test 데이터
mnist_test = dsets.MNIST(root='MNIST_data/',
                        train=False,
                        transform=transforms.ToTensor(),
                        download=True)

In [0]:
# 모델에 데이터를 쉽게 넣기위해 loader를 만들어줍니다.
data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                         batch_size=batch_size,
                                         shuffle=True,
                                         drop_last=True)

In [0]:
# 4. Model
class CNN(nn.Module):
    
    def __init__(self):
        super(CNN, self).__init__()
        
        # 1번 Conv Layer
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 32, 3, 1, 1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        
        # 2번 Conv Layer
        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 64, 3, 1, 1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        
        # 3번 Conv Layer
        self.layer3 = nn.Sequential(
            nn.Conv2d(64, 128, 3, 1, 1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        
        # 4번 FC Layer
        self.fc1 = nn.Linear(3*3*128, 625)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(625, 10, bias=True)
        
        # 가중치 초기화: Conv층은 Feature만 추출하고 실질적인 학습은 FC층에서 이루어집니다.
        torch.nn.init.xavier_uniform_(self.fc1.weight)
        torch.nn.init.xavier_uniform_(self.fc2.weight)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        
        out = out.view(out.size(0), -1) # FC에 넣기위에 size를 맞춰줍니다. (batch, channel, height, width) => (batch, channel*height*width)
        out = self.fc1(out)
        out = self.relu(out)
        out = self.fc2(out)
        return out

In [0]:
model = CNN().to(device)
model # 모델 구조를 확인합니다.

CNN(
  (layer1): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer3): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc1): Linear(in_features=1152, out_features=625, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=625, out_features=10, bias=True)
)

In [0]:
# 모델 테스트: 임의의 Tensor를 넣어서 사이즈가 잘 나오는지 확인합니다.
value = torch.Tensor(1, 1, 28, 28).to(device)
print(model(value).shape)

torch.Size([1, 10])


In [0]:
# 5. Loss, Optimizer
criterion = nn.CrossEntropyLoss().to(device) # loss 지정
optimizer = optim.Adam(model.parameters(), lr=learning_rate) # optimizer 지정

In [0]:
# 6. Train
total_batch = len(data_loader) # 전체 배치 개수
print('Learning Started!')

for epoch in range(training_epochs): # 15 에폭
    avg_cost = 0 # 각 에폭의 loss
    
    for X, Y in data_loader:
        X = X.to(device)
        Y = Y.to(device)
        
        optimizer.zero_grad() # 
        hypothesis = model(X) # CNN을 돌렸을때 예측값(y)
        
        cost = criterion(hypothesis, Y) # 예측값과 실제값 사이의 Loss 계산
        cost.backward() # Backpropa로 학습
        optimizer.step() # Optimizer
        
        avg_cost += cost / total_batch # 각 batch의 cost값을 모두 더한 후 전체 batch로 나눠 1에폭의 cost를 구한다(전체 batch의 평균값)
        
    print('[Epoch:{}] cost = {}'.format(epoch+1, avg_cost))
    
print('Learning Finished!')

Learning Started!
[Epoch:1] cost = 0.17660583555698395
[Epoch:2] cost = 0.041594985872507095
[Epoch:3] cost = 0.030163023620843887
[Epoch:4] cost = 0.022394094616174698
[Epoch:5] cost = 0.017288245260715485
[Epoch:6] cost = 0.014867338351905346
[Epoch:7] cost = 0.012478440999984741
[Epoch:8] cost = 0.010354137048125267
[Epoch:9] cost = 0.009000550955533981
[Epoch:10] cost = 0.006202113814651966
[Epoch:11] cost = 0.007943510077893734
[Epoch:12] cost = 0.007082311902195215
[Epoch:13] cost = 0.0065126265399158
[Epoch:14] cost = 0.004620209336280823
[Epoch:15] cost = 0.0062331282533705235
Learning Finished!


In [0]:
# 7. Test
with torch.no_grad():
    X_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device) # test Data
    Y_test = mnist_test.test_labels.to(device) # test label
    
    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test # 예측값과 내 값이 맞는지 확인
    accuracy = correct_prediction.float().mean()
    print("Accuracy:", accuracy.item())

Accuracy: 0.9888999462127686


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:58: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:48: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")
